In [2]:
import pandas as pd
import numpy as np

In [3]:
auctions = pd.read_csv('train_auctions.csv.gzip')

In [48]:
auctions.head().T

,0,1,2,3,4
created,2019-03-13 01:43:33.445,2019-03-13 04:58:35.078,2019-03-13 04:20:57.666,2019-03-13 04:20:57.698,2019-03-10 22:24:56.684
application_id,0,0,0,0,0
ref_type,1891515180541284343,1891515180541284343,1891515180541284343,1891515180541284343,1891515180541284343
ref_hash,8464844987297247076,3250564871270161533,7953318831018100268,7953318831018100268,8355110941557237501
click_hash,NaN,NaN,NaN,NaN,NaN
attributed,False,False,False,False,False
implicit,False,False,True,False,True
device_countrycode,6333597102633388268,6333597102633388268,6333597102633388268,6333597102633388268,6333597102633388268
device_brand,3.08306e+17,3.08306e+17,5.13799e+17,5.13799e+17,1.08337e+18
device_model,4.44525e+18,4.44525e+18,6.02654e+18,6.02654e+18,7.87661e+18


In [5]:
auctions.dtypes

created                object
application_id          int64
ref_type                int64
ref_hash                int64
click_hash            float64
attributed               bool
implicit                 bool
device_countrycode      int64
device_brand          float64
device_model          float64
session_user_agent     object
user_agent             object
event_uuid             object
kind                   object
wifi                   object
trans_id               object
ip_address              int64
device_language       float64
dtype: object

In [8]:
len(auctions)

3412

In [9]:
auctions.describe()

,application_id,ref_type,ref_hash,click_hash,device_countrycode,device_brand,device_model,ip_address,device_language
count,3412.000000,3.412000e+03,3.412000e+03,0.0,3.412000e+03,1.047000e+03,3.411000e+03,3.412000e+03,3.378000e+03
mean,10.247655,1.736533e+18,4.598202e+18,NaN,4.764401e+18,1.877864e+18,4.245276e+18,4.562040e+18,4.915828e+18
std,6.016194,1.936976e+17,2.663828e+18,NaN,1.678051e+18,1.886461e+18,2.620844e+18,2.683422e+18,2.477019e+18
min,0.000000,1.494519e+18,4.958527e+15,NaN,2.970471e+18,3.083059e+17,5.112505e+16,7.118754e+15,1.101539e+17
25%,7.000000,1.494519e+18,2.268353e+18,NaN,2.970471e+18,3.083059e+17,2.331947e+18,2.249197e+18,3.301378e+18
50%,9.000000,1.891515e+18,4.603807e+18,NaN,6.333597e+18,2.208835e+18,2.983638e+18,4.436113e+18,3.301378e+18
75%,10.000000,1.891515e+18,6.912404e+18,NaN,6.333597e+18,2.523246e+18,6.882415e+18,7.017564e+18,6.977049e+18
max,36.000000,1.891515e+18,9.218934e+18,NaN,6.333597e+18,9.169527e+18,9.222450e+18,9.221950e+18,9.028383e+18


In [13]:
#auctions.memory_usage(deep=True)

In [12]:
auctions.isnull().sum()

created                  0
application_id           0
ref_type                 0
ref_hash                 0
click_hash            3412
attributed               0
implicit                 0
device_countrycode       0
device_brand          2365
device_model             1
session_user_agent      48
user_agent            1683
event_uuid            2547
kind                  2547
wifi                  1683
trans_id              3406
ip_address               0
device_language         34
dtype: int64

In [14]:
auctions['device_language'].value_counts()

3.301378e+18    1484
9.028383e+18     714
4.621024e+18     632
6.977049e+18     238
4.077062e+17      70
1.526421e+18      56
4.060930e+18      48
2.822843e+17      39
6.035180e+18      24
1.193279e+18      21
6.535228e+18      10
1.101539e+17       9
3.177265e+18       5
3.695787e+17       5
3.095856e+18       4
6.569908e+18       2
5.846217e+18       2
7.774316e+18       2
4.594650e+18       2
6.782349e+18       1
5.061642e+18       1
1.686906e+18       1
5.794427e+17       1
2.488366e+18       1
3.544544e+18       1
3.074904e+18       1
2.061937e+18       1
4.073835e+18       1
6.846873e+18       1
7.045130e+18       1
Name: device_language, dtype: int64

In [15]:
auctions['device_brand'].value_counts()

3.083059e+17    472
2.208835e+18    185
2.523246e+18    116
5.951325e+18     72
2.987569e+18     40
3.812621e+18     37
5.137992e+17     34
6.538562e+18     26
3.228516e+18      9
1.083369e+18      9
4.371308e+18      7
3.093166e+18      5
2.262848e+18      5
4.994101e+18      4
3.849490e+18      4
8.285870e+18      3
3.742387e+18      3
5.645692e+18      3
8.230045e+17      3
6.111647e+18      3
9.169527e+18      1
5.588755e+18      1
3.172231e+18      1
5.189983e+18      1
3.624089e+18      1
7.855186e+18      1
2.748831e+18      1
Name: device_brand, dtype: int64

In [17]:
auctions['ref_type'].value_counts()

1891515180541284343    2080
1494519392962156891    1332
Name: ref_type, dtype: int64

In [19]:
#auctions['ref_hash'].value_counts()

In [42]:
auctions.loc[auctions['device_model'].isnull()]

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
2157,2019-03-09 16:08:19.919,30,1494519392962156891,4364935818432173881,NaN,False,True,6333597102633388268,NaN,NaN,HasOffers Mobile AppTracking v1.0,NaN,78082c4c-0fe1-4fc7-a718-4eba1d943b9c,Open,NaN,NaN,2208800741316080623,NaN


In [47]:
auctions['device_model'].describe()

count    3.411000e+03
mean     4.245276e+18
std      2.620844e+18
min      5.112505e+16
25%      2.331947e+18
50%      2.983638e+18
75%      6.882415e+18
max      9.222450e+18
Name: device_model, dtype: float64

In [55]:
#auctions['device_model'].value_counts()

In [46]:
#auctions.loc[auctions['device_language'].isnull()]

In [51]:
#auctions.loc[auctions['session_user_agent'].isnull()]
auctions['session_user_agent'].describe()

count             3364
unique              12
top       http-kit/2.0
freq              1729
Name: session_user_agent, dtype: object

In [53]:
auctions['session_user_agent'].value_counts()

http-kit/2.0                                                                                                                                                                                                                                                                  1729
adjust.com                                                                                                                                                                                                                                                                    1592
HasOffers Mobile AppTracking v1.0                                                                                                                                                                                                                                               22
Mozilla/5.0 (iPhone; CPU iPhone OS 12_1_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0 Mobile/15E148 Safari/604.1                                      